In [2]:
import numpy
import pandas as pd
# Основная выборка, разбитая на train и test

train = pd.read_csv('Public/train.csv', sep=';')
test = pd.read_csv('Public/test.csv', sep=';')

# Дополнительные данные 

ed = pd.read_csv('Public/education.csv', sep=';')
ws = pd.read_csv('Public/worldskills.csv', sep=';')
em = pd.read_csv('Public/employements.csv', sep=';')
em_multi = pd.read_csv('Public/employements_mult.csv', sep=';')
ed_multi = pd.read_csv('Public/education_mult.csv', sep=';')


train = train.merge(ed, left_on='id', right_on='id', how = 'left')
train = train.merge(em, left_on='id', right_on='id', how = 'left')
#train = train.merge(ws, left_on='id', right_on='id')

/home/pinkurchin/anaconda2/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
today_dt = pd.to_datetime('20.12.2020', format='%d.%m.%Y')
# количество дней на каждом месте
em_multi['start_date'] = em_multi['start_date'].fillna(value = today_dt)
em_multi['start_date'] = pd.to_datetime(em_multi['start_date'], format='%Y-%m-%d', errors = 'coerce')

em_multi['finish_date'] = em_multi['finish_date'].fillna(value = today_dt)
em_multi['finish_date'] = pd.to_datetime(em_multi['finish_date'], format='%Y-%m-%d', errors = 'coerce')
em_multi['days_at_th_work'] = em_multi['finish_date'] - em_multi['start_date']
em_multi['days_at_th_work'] = em_multi['days_at_th_work'].apply(lambda x: x.days)


In [444]:
#агрегируем. Считаем количество мест, среднее количество дней на одном мес те
from functools import reduce
em_multi_1 = em_multi[['id', 'employer']].groupby(['id'], as_index=False).agg('count')
em_multi_2 = em_multi[['id', 'days_at_th_work']].groupby(['id'], as_index=False).agg('mean')
em_multi_2a = em_multi[['id', 'days_at_th_work']].groupby(['id'], as_index=False).agg('sum')
em_multi_2a = em_multi_2a.rename(columns = {'days_at_th_work' : 'total_work_days'})
em_multi_3 = em_multi[['id', 'start_date']].groupby(['id'], as_index=False).agg('min')
em_multi_4 = em_multi[['id', 'finish_date']].groupby(['id'], as_index=False).agg('max')
em_multi_ls = [em_multi_1, em_multi_2, em_multi_2a, em_multi_3, em_multi_4]
    
#объединяем все нагрегированные датасеты в один большой
em_multi_gr = reduce(lambda  left,right: pd.merge(left,right,on=['id'], how='right'), em_multi_ls)
em_multi_gr = em_multi_gr.rename(columns = {'employer': 'emp_count', 'finish_date' : 'total_finish_date', 'start_date' :  'total_start_date'})

#могли быть перерывы между работами
#em_multi_gr['total_work_days'] = em_multi_gr['total_finish_date'] - em_multi_gr['total_start_date']
#em_multi_gr['total_work_days'] = em_multi_gr['total_work_days'].apply(lambda x: x.days)

In [445]:
em_multi[em_multi.id == 4]

,id,position,employer,achievements,responsibilities,start_date,finish_date,days_at_th_work
679674,4.0,Специалист 3 категории,"ГБУ СО ""МФЦ""",<p>Награждена Почетной грамотой администрацией...,<p>Предоставление заявителям государственных и...,2015-07-01,2020-09-01,1889.0
679675,4.0,Старший специалист по привлечению клиентов,"ОАО Банк ""Открытие""",<p>Расширила базу клиентов</p>,"<p>Обслуживание физических лиц, активное привл...",2011-07-01,2013-01-01,550.0
679676,4.0,Кредитный специалист,"ЗАО ""Кредит Европа Банк""",<p>Получила награду за превосходное обслуживан...,"<p>Обслуживание физических лиц, первичная пров...",2013-01-01,2015-04-01,820.0


In [426]:
em_multi_gr.head()

,id,emp_count,days_at_th_work,total_work_days,total_start_date,total_finish_date
0,0.0,4,304.750000,1219.0,2017-05-01,2020-04-01
1,1.0,1,446.000000,446.0,2019-10-01,2020-12-20
2,2.0,1,30.000000,30.0,2014-06-01,2014-07-01
3,3.0,4,1177.000000,4708.0,2005-09-01,2020-12-20
4,4.0,3,1086.333333,3259.0,2011-07-01,2020-09-01


In [446]:
#Образование .сколько, какое, москва ли?
ed_multi['institution'] = ed_multi['institution'].fillna(value="")
ed_multi['institution'] = ed_multi['institution'].apply(lambda x: str(x).lower())
ed_multi['high_ed'] = ed_multi['institution'].apply(lambda x: 1 if (('универ' in x) or ('академ' in x) or ('инстит' in x) or 
                                                              ('высш' in x) or ('консерв' in x)) else 0)
ed_multi['voc_ed'] = ed_multi['institution'].apply(lambda x: 1 if (('учил' in x) or ('техник' in x) or ('колледж' in x) or 
                                                              ('ссуз' in x) or ('пту' in x)) else 0);
ed_multi['postgr_ed'] = ed_multi['institution'].apply(lambda x: 1 if (('аспиран' in x) or ('докторан' in x) or ('интернату' in x)) else 0)
ed_multi['Mosc'] = ed_multi['institution'].apply(lambda x: 1 if ('моск' in x) else 0 )
ed_multi_gr = ed_multi[['id', 'high_ed', 'voc_ed', 'postgr_ed', 'Mosc']].groupby(['id'], as_index=False).agg('sum')



In [ ]:
ed

In [451]:
train_delta = ed_multi_gr
train_delta = train_delta.merge(em_multi_gr, left_on='id', right_on='id', how='left')
train_delta['emp_count'] = train_delta['emp_count'].fillna(value = 0)
train_delta['days_at_th_work'] = train_delta['days_at_th_work'].fillna(value = 0)
train_delta['total_work_days'] = train_delta['total_work_days'].fillna(value = 0)
import datetime
today_dt = pd.to_datetime('20.12.2020', format='%d.%m.%Y')
train_delta['total_start_date'] = train_delta['total_start_date'].fillna(value = today_dt)
train_delta['total_finish_date'] = train_delta['total_finish_date'].fillna(value = today_dt)


In [453]:
train_delta.shape


(375386, 10)

In [396]:
train = train.merge(ed_multi_gr, left_on = 'id', right_on='id', how = 'left')
print (train[(train.education_type_le != 3) and (traingh > 0)])

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [238]:
train.columns

Index(['id', 'position_x', 'region', 'industry', 'locality', 'locality_name',
       'education_type', 'drive_licences', 'citizenship', 'schedule',
       'employement_type', 'age', 'gender', 'experience', 'salary_desired',
       'relocation_ready', 'travel_ready', 'retraining_ready',
       'is_worldskills_participant', 'has_qualifications', 'completeness_rate',
       'creation_date', 'modification_date', 'publish_date', 'salary',
       'position_y', 'employer', 'achievements', 'responsibilities',
       'start_date', 'finish_date'],
      dtype='object')

In [239]:
print(train.industry)

0         Государственная служба, некоммерческие организ...
1         Государственная служба, некоммерческие организ...
2                 Административная работа, секретариат, АХО
3         Государственная служба, некоммерческие организ...
4         Здравоохранение, спорт, красота, социальное об...
                                ...                        
306265                Продажи, закупки, снабжение, торговля
306266    Государственная служба, некоммерческие организ...
306267                                        Юриспруденция
306268                                         Производство
306269                    Работы, не требующие квалификации
Name: industry, Length: 306270, dtype: object


In [215]:
# Смержим с массивом по ЗП
real_salary_df = pd.read_excel('OffSal2019_2020_2.xlsx', header=1)
train = train.merge(real_salary_df, left_on='region', right_on='Region', how='left' )



In [211]:
train.ix[200, :]

/home/pinkurchin/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


id                                                                  280
position_x                                                    Бухгалтер
region                                                        г. Москва
industry              Финансы, кредит, страхование, пенсионное обесп...
locality                                                  7700000000000
                                            ...                        
schedule_part_time                                                    0
schedule_flex                                                         0
schedule_shift                                                        0
schedule_big_shift                                                    0
schedule_irregular                                                    0
Name: 200, Length: 70, dtype: object

In [212]:
train.dtypes

id                     int64
position_x            object
region                object
industry              object
locality               int64
                       ...  
schedule_part_time     int64
schedule_flex          int64
schedule_shift         int64
schedule_big_shift     int64
schedule_irregular     int64
Length: 70, dtype: object

In [196]:
train['start_date'].isnull().sum()

0

In [216]:
from datetime import datetime
import datetime

#for date
train['creation_date'] = pd.to_datetime(train['creation_date'], format='%Y-%m-%d')
train['create_year'] = pd.DatetimeIndex(train['creation_date']).year
train["create_month"]= pd.DatetimeIndex(train["creation_date"]).month
train["create_day"] = pd.DatetimeIndex(train["creation_date"]).day
train["create_dt_half_m"] =  train["create_day"].apply(lambda x : 2 if (x > 15) else 1)    


train['modification_date'] = pd.to_datetime(train['modification_date'], format='%Y-%m-%d')
train['modification_year'] = pd.DatetimeIndex(train['modification_date']).year
train["modification_month"]= pd.DatetimeIndex(train["modification_date"]).month
train["modification_day"] = pd.DatetimeIndex(train["modification_date"]).day
train["modification_dt_half_m"] =  train["create_day"].apply(lambda x : 2 if (x > 15) else 1)


train['publish_date'] = pd.to_datetime(train['publish_date'], format='%Y-%m-%d')
train['publish_year'] = pd.DatetimeIndex(train['publish_date']).year
train["publish_month"]= pd.DatetimeIndex(train["publish_date"]).month
train["publish_day"] = pd.DatetimeIndex(train["publish_date"]).day
train["publish_dt_half_m"] =  train["publish_day"].apply(lambda x : 2 if (x > 15) else 1)    





In [457]:
unique_numbers = list(set(ed_multi.id))
print (len(unique_numbers))

391077


In [78]:
train.drive_licences.isnull().sum()

284

In [393]:
#Образование
ed_dict = {'Среднее профессиональное' : 0, 'Неоконченное высшее' : 1, 'Среднее' : 2, 'Высшее' : 3}
train['education_type_le'] = train['education_type'].replace(ed_dict)


In [208]:
#MultiLabelBinarizer
train['drive_licences_str'] = train['drive_licences'].str.replace(" ", "")
train['drive_licences_A'] = train['drive_licences_str'].apply(lambda x: 1 if ('A' in str(x)) else 0);                                                              
train['drive_licences_B'] = train['drive_licences_str'].apply(lambda x: 1 if ('B' in str(x)) else 0);                                                              
train['drive_licences_C'] = train['drive_licences_str'].apply(lambda x: 1 if ('C' in str(x)) else 0);                                                              
train['drive_licences_D'] = train['drive_licences_str'].apply(lambda x: 1 if ('D' in str(x)) else 0);                                                                                                                            
train['drive_licences_E'] = train['drive_licences_str'].apply(lambda x: 1 if ('E' in str(x)) else 0);


In [209]:
# отсортировано по ВВП на деше населения
citiz_dict = {'Российская Федерация' : 12,'Республика Казахстан' : 11,  'Республика Беларусь' : 10, 'Туркменистан' : 9, 
'Азербайджанская Республика' : 8, 'Республика Армения':7, 'Украина' : 6, 'Республика Молдова' : 5, 
   'Республика Узбекистан' : 4, 'Кыргызская Республика' : 3, 'Республика Таджикистан' : 2, 'Иное' : 1 }
train['citizenship_le'] = train['citizenship'].replace(citiz_dict)

In [210]:

train['schedule_full_time'] = train['schedule'].apply(lambda x: 1 if ('Полный рабочий день' in str(x)) else 0);                                                              
train['schedule_part_time'] = train['schedule'].apply(lambda x: 1 if ('Неполный рабочий день' in str(x)) else 0);                                                              
train['schedule_flex'] = train['schedule'].apply(lambda x: 1 if ('Гибкий график' in str(x)) else 0);                                                              
train['schedule_shift'] = train['schedule'].apply(lambda x: 1 if ('Сменный график' in str(x)) else 0);                                                                                                                            
train['schedule_big_shift'] = train['schedule'].apply(lambda x: 1 if ('Вахтовый метод' in str(x)) else 0);                                                                                                                        
train['schedule_irregular'] = train['schedule'].apply(lambda x: 1 if ('Ненормированный рабочий день' in str(x)) else 0);

In [165]:


print (train[train.days_since_lockdown == 0]['days_since_lockdown'])

2         0.0
5         0.0
10        0.0
12        0.0
13        0.0
         ... 
306255    0.0
306257    0.0
306260    0.0
306263    0.0
306267    0.0
Name: days_since_lockdown, Length: 107674, dtype: float64


In [264]:
train.columns

Index(['id', 'position_x', 'region', 'industry', 'locality', 'locality_name',
       'education_type', 'drive_licences', 'citizenship', 'schedule',
       'employement_type', 'age', 'gender', 'experience', 'salary_desired',
       'relocation_ready', 'travel_ready', 'retraining_ready',
       'is_worldskills_participant', 'has_qualifications', 'completeness_rate',
       'creation_date', 'modification_date', 'publish_date', 'salary',
       'graduation_year', 'institution', 'description', 'position_y',
       'employer', 'achievements', 'responsibilities', 'start_date',
       'finish_date', 'graduate_age', 'finish_winter', 'finish_spring',
       'finish_summer', 'finish_autumn', 'start_winter', 'start_spring',
       'start_summer', 'start_autumn'],
      dtype='object')

In [284]:

train_delta = train[['id',
'graduate_age', 'finish_winter', 'finish_spring',
       'finish_summer', 'finish_autumn', 'start_winter', 'start_spring',
       'start_summer', 'start_autumn']]

In [285]:
train_delta.columns


Index(['id', 'graduate_age', 'finish_winter', 'finish_spring', 'finish_summer',
       'finish_autumn', 'start_winter', 'start_spring', 'start_summer',
       'start_autumn'],
      dtype='object')

In [454]:

train_v4 = pd.read_csv('train_v8.csv', sep=',')
train_v5 = train_v4;
train_v5 = train_v4.merge(train_delta, left_on='id', right_on='id', how='left')
train_v5.to_csv('train_v9', sep = ',', index = False)

In [456]:
print(train_delta.isnull().sum())
print(train_v4.shape)
print(train_v5.shape)
print(train_delta.shape)

id                   0
high_ed              0
voc_ed               0
postgr_ed            0
Mosc                 0
emp_count            0
days_at_th_work      0
total_work_days      0
total_start_date     0
total_finish_date    0
dtype: int64
(306270, 122)
(306270, 131)
(375386, 10)


In [437]:
import datetime


train['creation_date'] = pd.to_datetime(train['creation_date'], format='%Y-%m-%d')
train['modification_date'] = pd.to_datetime(train['modification_date'], format='%Y-%m-%d')
today_dt = pd.to_datetime('20.12.2020', format='%d.%m.%Y')
train['start_date'] = train['start_date'].fillna(value = today_dt)
train['start_date'] = pd.to_datetime(train['start_date'], format='%Y-%m-%d', errors = 'coerce')
train['start_year'] = pd.DatetimeIndex(train['start_date']).year
train["start_month"]= pd.DatetimeIndex(train["start_date"]).month
train["start_day"] = pd.DatetimeIndex(train["start_date"]).day
train["start_dt_half_m"] =  train["start_day"].apply(lambda x : 2 if (x > 15) else 1)
train["start_quarter"] =  train["start_month"].apply(lambda x : x // 4);

train['finish_date'] = train['finish_date'].fillna(value = today_dt)
train['finish_date'] = pd.to_datetime(train['finish_date'], format='%Y-%m-%d',  errors = 'coerce')
train['finish_year'] = pd.DatetimeIndex(train['finish_date']).year
train["finish_month"]= pd.DatetimeIndex(train["finish_date"]).month
train["finish_day"] = pd.DatetimeIndex(train["finish_date"]).day
train["finish_dt_half_m"] =  train["finish_day"].apply(lambda x : 2 if (x > 15) else 1)
train["finish_quarter"] =  train["finish_month"].apply(lambda x : x // 4);

lockdown_dt = pd.to_datetime('27.03.2020', format='%d.%m.%Y')
delta = datetime.timedelta(seconds=1)
train['days_bt_lockdown_cr'] =  train['creation_date'].apply(lambda x:max(x - lockdown_dt,  delta).days)
train['days_bt_lockdown_mod'] =  train['modification_date'].apply(lambda x:max(x - lockdown_dt,  delta).days)
train['days_at_last_work'] = (train['finish_date'] - train['start_date'])
train['days_at_last_work'] = train['days_at_last_work'].apply(lambda x: x.days)

In [283]:
train['graduate_age'] = train['graduation_year'] - (pd.DatetimeIndex(train['modification_date']).year - train['age'])

train['start_date'] = train['start_date'].fillna(value = today_dt)
train['start_date'] = pd.to_datetime(train['start_date'], format='%Y-%m-%d', errors = 'coerce')
train['finish_date'] = train['finish_date'].fillna(value = today_dt)
train['finish_date'] = pd.to_datetime(train['finish_date'], format='%Y-%m-%d',  errors = 'coerce')
train['finish_winter'] = train['finish_date'].apply(lambda x: 1 if ((x.month < 3) or (x.month == 12)) else 0)
train['finish_spring'] = train['finish_date'].apply(lambda x: 1 if ((x.month > 2) and (x.month < 6)) else 0)
train['finish_summer'] = train['finish_date'].apply(lambda x: 1 if ((x.month > 5) and (x.month < 9)) else 0)
train['finish_autumn'] = train['finish_date'].apply(lambda x: 1 if ((x.month > 8) and (x.month < 12)) else 0)

train['start_winter'] = train['start_date'].apply(lambda x: 1 if ((x.month < 3) or (x.month == 12)) else 0)
train['start_spring'] = train['start_date'].apply(lambda x: 1 if ((x.month > 2) and (x.month < 6)) else 0)
train['start_summer'] = train['start_date'].apply(lambda x: 1 if ((x.month > 5) and (x.month < 9)) else 0)
train['start_autumn'] = train['start_date'].apply(lambda x: 1 if ((x.month > 8) and (x.month < 12)) else 0)

In [276]:
print(train['graduate_age'])

0         23.0
1         22.0
2         19.0
3         19.0
4          NaN
          ... 
306265    16.0
306266    22.0
306267    24.0
306268    26.0
306269    18.0
Name: graduate_age, Length: 306270, dtype: float64


In [287]:
train_v5.shape

(306270, 104)